In [16]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

# load dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [104]:
classes = set(y_train)
variousSamples = list()
for i in classes:
  ind = np.where(y_train == i)[0][0]
  print(ind)
  variousSamples.append(x_train[ind])

1
16
5
3
19
8
18
6
23
0


In [105]:
classes

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [79]:
x_flatten_train = x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2],1)
y_encoded = np.zeros((y_train.shape[0], max(classes) + 1))
y_encoded[np.arange(y_train.shape[0]), y_train] = 1

In [85]:
y_encoded = y_encoded.reshape(60000,10,1)

In [84]:
y_encoded

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
layers = 3
samples = y_train.shape[0]
epochs = 5
nodesPerLayer = list()

In [20]:
nodesPerLayer.append(784)
for i in range(0,layers):
  nodesPerLayer.append(int(512/(2**(i+1))))
nodesPerLayer.append(10)

In [21]:
print(nodesPerLayer)

[784, 256, 128, 64, 10]


In [22]:
weights = list()
bias = list()
for i in range(1,len(nodesPerLayer)):
  w = np.random.rand(nodesPerLayer[i],nodesPerLayer[i-1])
  b = np.random.rand(nodesPerLayer[i],1)
  weights.append(w)
  bias.append(b)

In [101]:
def func(activation,a_k):
  #print(a_k)
  if(activation == "tanh"):
    a_k = np.exp(a_k) - 1/np.exp(a_k) / np.exp(a_k) + 1/np.exp(a_k)
  else:
    a_k = (1 / (1 + 1/np.exp(a_k)))
  return a_k

In [24]:
def derivativeFun(activation,a_k):
  activationResult = func(activation,a_k)
  if(activation == "tanh"):
    activationResult = 1 - (activationResult**2)
  else:
    activationResult = activationResult - (activationResult**2)
  return activationResult

In [25]:
def decision(a_k,classificationFunction):
  if classificationFunction == "crossEntropy":
     a_k = np.exp(a_k)
     a_k = a_k / sum(a_k)
  return a_k

In [97]:
def OneHotEncode(C):
  oneHot = np.zeros(C.shape)
  oneHot[np.argmax(C)] = 1
  return oneHot

In [91]:
def forwardProp(inputX,activation,classificationFunction,weights,bias):
  h_k = inputX
  PreActivations = list()
  PostActivations = list()
  PostActivations.append(h_k)
  for k in range(0,layers):
    a_k = bias[k] + np.matmul(weights[k],h_k)
    PreActivations.append(a_k)
    h_k = func(activation,a_k)
    PostActivations.append(h_k)
  a_k = bias[layers] + np.matmul(weights[layers],h_k)
  PreActivations.append(a_k)
  yPred = decision(a_k,classificationFunction)
  return PreActivations,PostActivations,yPred

In [56]:
def backProp(pred,real,h_k,weights,activation,PreActivations):
  a_l_L_theta = -(real - pred)
  currentActivationGradient = a_l_L_theta
  WeightGradients = list()
  for i in range(layers,-1,-1):
    W_i_L_theta = np.matmul(currentActivationGradient,np.transpose(h_k[i-1]))
    WeightGradients.insert(0,W_i_L_theta)
    b_k_L_theta = a_l_L_theta
    h_i_prev_L_theta = np.matmul(np.transpose(weights[i]),currentActivationGradient)
    currentActivationGradient = h_i_prev_L_theta*derivativeFun(activation,PreActivations[i-1])
  return WeightGradients

In [ ]:
for i in range(0,epochs):
  for j in range(0,samples):
    A,B,C = forwardProp(x_flatten_train[j],"sigmoid","crossEntropy",weights,bias)
    newC = OneHotEncode(C)
    delta = backProp(newC,y_encoded[j],B,weights,"sigmoid",A)

In [75]:
newC.shape

(10, 1)